In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio
import dask.dataframe as dd

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [4]:
#notebook complete indicator
notebook = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = notebook, autoplay = True)

In [5]:
map = gpd.read_file('../../Data/Taxi Zones Shape/taxi_zones.shp').to_crs('epsg:4326')
map = map.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

In [6]:
df=dd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-03.csv', assume_missing=True)

df.head()

df.info()

df2=df.head(100)

df2.columns

In [7]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x[' dropoff_longitude']), 
                                             float(x[' dropoff_latitude']))), 
                                             axis=1,meta=(None, 'Float64'))

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 0.00951528549194336 seconds ---


In [8]:
start_time = time.time()

df['pickup_geometry'] = df.apply(lambda x: Point((float(x[' pickup_longitude']), 
                                             float(x[' pickup_latitude']))), 
                                             axis=1,meta=(None, 'Float64'))

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 0.010009527206420898 seconds ---


In [9]:
df.head(3)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_geometry,pickup_geometry
0,CMT,2014-03-01 01:07:38,2014-03-01 01:16:26,1.0,2.0,-73.951504,40.714113,1.0,N,-73.944278,40.690677,CRD,9.0,0.5,0.5,2.0,0.0,12.0,POINT (-73.94427799999998 40.690677),POINT (-73.951504 40.714113)
1,CMT,2014-03-01 01:08:03,2014-03-01 01:12:51,2.0,1.2,-74.004735,40.742173,1.0,N,-73.992228,40.748210,CRD,6.0,0.5,0.5,1.0,0.0,8.0,POINT (-73.99222799999998 40.74821),POINT (-74.00473499999998 40.742173)
2,CMT,2014-03-01 01:08:51,2014-03-01 01:13:18,3.0,0.5,-73.949302,40.707569,1.0,N,-73.951654,40.714250,CRD,5.0,0.5,0.5,1.2,0.0,7.2,POINT (-73.951654 40.71425),POINT (-73.949302 40.707569)


In [ ]:
start_time = time.time()

df3=df.compute()

start_time = time.time()

In [ ]:
df2_start=df[[' passenger_count',' trip_distance',' fare_amount',' mta_tax',' tip_amount',' tolls_amount','pickup_geometry']]

In [ ]:
df2_end=df[['dropoff_geometry']]

In [ ]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

In [ ]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

In [ ]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

In [ ]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

In [ ]:
geo_processing_start

In [ ]:
group_start = geo_processing_start.groupby(['LocationID']).describe()

In [ ]:
group_start.columns = ['_'.join(col) for col in group_start.columns.values]

group_start.to_csv('../../Data/yellow_tripdata_2014-2-start-processed.csv')

In [ ]:
group_end = geo_processing_end.value_counts(['LocationID'])

In [ ]:
group_end = pd.DataFrame(group_end,columns=['dropoff_count'])

group_end.to_csv('../../Data/yellow_tripdata_2014-2-end-processed.csv')

In [ ]:
merge=group_start.merge(group_end, how='left', on='LocationID')

In [ ]:
#REMEMBER TO RENAME CSV BEFORE START
merge.to_csv('../../Data/yellow_tripdata_2014-03-processed_Dask.csv')
complete

In [ ]:
print("--- %s seconds ---" % (time.time() - full_time))